<a href="https://colab.research.google.com/github/LisethTiria/Biosenales-2025-1/blob/main/practica_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

def calcular_rms(x):
    """
    Calcula el valor cuadrático medio (RMS) de una señal x.

    Parámetros:
        x (list o np.array): Señal discreta, una lista o arreglo de N muestras.

    Retorna:
        float: Valor RMS de la señal.
    """
    x = np.array(x)  # Asegura que sea un arreglo de NumPy
    N = len(x)
    rms = np.sqrt(np.sum(x**2) / N)
    return rms


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# prompt: quiero cargar un archivo de drive llamado signals.mat

from google.colab import drive
import scipy.io as sio
import numpy as np

drive.mount('/content/drive')

# Load the .mat file
try:
  mat_data = sio.loadmat('/content/drive/MyDrive/Colab Notebooks/Biosenales/signals.mat') # Replace with the correct path in your Google Drive
except FileNotFoundError:
  print("Error: 'signals.mat' not found. Please check the file path.")
  exit()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error: 'signals.mat' not found. Please check the file path.


In [4]:
import numpy as np
Fm=1024  # Frecuencia de muestreo en Hz
duracion = 30         # Duración de la señal en segundos
N = Fm * duracion     # Número total de muestras

t = np.linspace(0, duracion, N, endpoint=False)  # Vector de tiempo
print(t)

[0.00000000e+00 9.76562500e-04 1.95312500e-03 ... 2.99970703e+01
 2.99980469e+01 2.99990234e+01]


In [5]:
# prompt: plotea desde mat_data: ECG_asRecording y ECG_filtered sobre el vector de tiempo t creado anteriormente y quitarle el offser a ECG_asRecording

import matplotlib.pyplot as plt

# Assuming ECG_asRecording and ECG_filtered are in mat_data
ECG_asRecording = mat_data['ECG_asRecording'][0]
ECG_filtered = mat_data['ECG_filtered'][0]

# Remove offset from ECG_asRecording
ECG_asRecording = ECG_asRecording - np.mean(ECG_asRecording)

# Plot the signals
plt.figure(figsize=(12, 6))
plt.plot(t, ECG_asRecording, label='ECG (Raw)')
plt.plot(t, ECG_filtered, label='ECG (Filtered)')

plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('ECG')
plt.legend()
plt.grid(True)
plt.show()


NameError: name 'mat_data' is not defined

In [ ]:
# prompt: Tome la señal ECG_asRecording del mat_data. De dicha señal, seleccione un ciclo cardiaco (0.8 segundos), grafique. Calcule el promedio, el valor RMS, la varianza y la desviación estándar del ciclo.
import scipy
from scipy import signal
from scipy.stats import levene, ttest_ind, mannwhitneyu, shapiro

# Find R peaks (you might need to adjust the threshold and other parameters)
# This is a simplified example and might not be accurate for all ECG signals
peaks, _ = scipy.signal.find_peaks(ECG_asRecording, height=0.8, distance=150) # Adjust height and distance

# Select a heart cycle (0.8 seconds) around the first peak
if len(peaks)>0:
  peak_index = peaks[0]  # Index of the first peak
  cycle_start = int(max(0, peak_index - 0.4 * Fm)) # inicio del ciclo
  cycle_end = int(min(len(ECG_asRecording), peak_index + 0.4 * Fm)) # fin del ciclo
  heart_cycle = ECG_asRecording[cycle_start:cycle_end]
  time_cycle = t[cycle_start:cycle_end]


  # Calculate statistics
  mean_cycle = np.mean(heart_cycle)
  rms_cycle = calcular_rms(heart_cycle)
  variance_cycle = np.var(heart_cycle)
  std_dev_cycle = np.std(heart_cycle)

  # Print the results
  print("Mean:", mean_cycle)
  print("RMS:", rms_cycle)
  print("Variance:", variance_cycle)
  print("Standard Deviation:", std_dev_cycle)

  # Plot the selected heart cycle
  plt.figure(figsize=(8, 4))
  plt.plot(time_cycle, heart_cycle)
  plt.xlabel('Tiempo (s)')
  plt.ylabel('Amplitud')
  plt.title('Ciclo Cardíaco')
  plt.grid(True)
  plt.show()
else:
    print("No peaks found in the ECG signal.")


In [ ]:
# prompt: A partir de la información suministrada en el archivo .mat, determine el tiempo de duración de las señales, cree un vector de tiempo.
# · Grafique la señal sin filtrar y filtrada, compárelas. Comente lo que observa, y describa cómo debe ser el comportamiento del filtro que se utilizó. Recuerde las frecuencias de interés de la señal ECG.
# · Tome la señal tal cual se registró. De dicha señal, seleccione un ciclo cardiaco, grafique. Calcule el promedio, el valor RMS, la varianza y la desviación estándar del ciclo. Analice los resultados.
# · Tome la señal filtrada. Repita el procedimiento anterior con esta señal, compare con los resultados obtenidos en el ítem anterior.
# · Extraiga 15 ciclos de la señal filtrada, y para cada ciclo calcule el valor promedio y la varianza, grafique cada ciclo, y analice (gráfica y estadísticamente). ¿Se puede considerar que la señal es estacionaria?
# · Realice dos veces comparaciones entre dos ciclos cualquiera utilizando una prueba t. Determine estadísticamente si existe diferencia estadística entre los ciclos, Compruebe los supuestos necesarios para realizar una prueba t, esto es: Normalidad de la variable, independencia (se asume que los grupos son independientes), y homocedasticidad (use una prueba de Levene), finalmente realice la prueba t para determinar si existen diferencias entre los ciclos. Qué indica este análisis respecto de la estacionariedad, explique. De no cumplirse los requisitos, realice entonces un análisis no paramétrico (prueba U de Mann-Whitney).
# · Compruebe los resultados anteriores aplicando una prueba de Dickey-Fuller a la señal (Estacionariedad)

import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import levene, ttest_ind, mannwhitneyu, shapiro
from statsmodels.tsa.stattools import adfuller

# Extract signals and sampling frequency
try:
    ecg_signal = mat_data['ecg_signal'][0]
    ecg_signal_filtered = mat_data['ecg_signal_filtered'][0]
    fs = mat_data['fs'][0][0]
except KeyError:
    print("Error: 'ecg_signal', 'ecg_signal_filtered', or 'fs' not found in the .mat file.")
    exit()

# Time vector
time = np.arange(0, len(ecg_signal) / fs, 1 / fs)

# Plot signals
plt.figure(figsize=(12, 6))
plt.plot(time, ecg_signal, label='Sin filtrar')
plt.plot(time, ecg_signal_filtered, label='Filtrada')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Comparación de señales ECG')
plt.legend()
plt.grid(True)
plt.show()

# Select a cardiac cycle
start_index =  # Replace with the start index of the cycle
end_index =  # Replace with the end index of the cycle
cycle = ecg_signal[start_index:end_index]
time_cycle = time[start_index:end_index]

# Calculate statistics for the selected cycle
mean_cycle = np.mean(cycle)
rms_cycle = calcular_rms(cycle)
variance_cycle = np.var(cycle)
std_cycle = np.std(cycle)


# ... (rest of the code for other items, similar to the above cycle analysis)


# Dickey-Fuller test
result = adfuller(ecg_signal_filtered)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

